In [1]:
import os

# Why Torch? You'll find the answer in the .md files! 
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
import keras
from torch.utils.data import DataLoader

from dataset.dataset_loader import dataset_loader
import keras_tuner

from models.augmented_dnn.augmented_dnn import TwoHiddenLayersTunableAugmentedDNN
from models.structure.tunable_model_family_hypermodel import TunableModelFamilyHypermodel
from utils.my_tuner import HistoryDeletingBayesianOptimization

from dataset.k_fold_dataset_wrapper import KFoldDatasetWrapper

In [3]:
import torch

torch.cuda.is_available()

True

In [4]:
# Initial steps
hyperparameters = keras_tuner.HyperParameters()
train, test = dataset_loader((192, 192), is_grayscale=False)
dataset_split_controller = KFoldDatasetWrapper(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)
train_dataloader = DataLoader(dataset=local_train, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, batch_size=32, shuffle=True)

In [5]:

hyperparameters.Fixed("lr", 1e-4)
hyperparameters.Fixed("momentum", 0.9)

# We add dropout later to improve the best networks
hyperparameters.Fixed("dropout_0", False)
hyperparameters.Fixed("dropout_1", False)

project_name = "two-layers-192-192"
project_directory = "augmented-dnn-search"

tuner = HistoryDeletingBayesianOptimization(
    TunableModelFamilyHypermodel((3, 192, 192), TwoHiddenLayersTunableAugmentedDNN()),
    hyperparameters=hyperparameters,
    objective='val_loss',
    tune_new_entries=True,
    executions_per_trial=1,
    overwrite=False,
    directory=project_directory,
    max_trials=15,
    project_name=project_name
)

In [ ]:
tuner.search(train_dataloader, epochs=20, validation_data=validation_dataloader, callbacks=[keras.callbacks.CSVLogger(
    f"{project_directory}/{project_name}/search.log", separator=",", append=True)
])

Trial 3 Complete [00h 15m 11s]
val_loss: 0.4819851815700531

Best val_loss So Far: 0.4819851815700531
Total elapsed time: 00h 15m 12s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
0.0001            |0.0001            |lr
0.9               |0.9               |momentum
False             |False             |dropout_0
False             |False             |dropout_1
1792              |1792              |units_0
1408              |1152              |units_1

Epoch 1/20
 47/119 ━━━━━━━━━━━━━━━━━━━━ 21s 295ms/step - accuracy: 0.5451 - loss: 0.7505